##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Neural machine translation with attention

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io
import time
import pandas as pd
import re

In [3]:
df= pd.read_excel('/content/gdrive/My Drive/kor.xlsx',sheet_name=1)

In [4]:
# Download the file
path_to_file = '/content/gdrive/My Drive/kor.xlsx'

In [5]:
# # Converts the unicode file to ascii
# def unicode_to_ascii(s):
#   return ''.join(c for c in unicodedata.normalize('NFD', s)
#       if unicodedata.category(c) != 'Mn')


# def preprocess_sentence(w):
#   w = w.lower().strip()

#   # creating a space between a word and the punctuation following it
#   # eg: "he is a boy." => "he is a boy ."
#   # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
#   w = re.sub(r"([?.!,¿])", r" \1 ", w)
#   w = re.sub(r'[" "]+', " ", w)
  
#   # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
#   w = re.sub(r"[^ㄱ-ㅣ가-힣a-zA-Z?.!,]+", " ", w)
#   # w = re.sub(r"[^ ㄱ-ㅣ가-힣?.!,¿]+", " ", w)
#   w = w.strip()

#   # adding a start and an end token to the sentence
#   # so that the model know when to start and stop predicting.
#   if len(re.findall('[ㄱ-ㅣ가-힣]+',w)) == 0:
#     w = '<start> ' + w + ' <end>'
#   return w

In [6]:
#################################reverse_진행###############################################
def preprocess_sentence(w):
  w = w.lower().strip()
  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^ㄱ-ㅣ가-힣a-zA-Z?.!,]+", " ", w)
  # w = re.sub(r"[^ ㄱ-ㅣ가-힣?.!,¿]+", " ", w)
  w = w.strip()
  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  if len(re.findall('[ㄱ-ㅣ가-힣]+',w)) == 0:
    w = '<start> ' + w + ' <end>'
  else :
    # reverse
    w = '<start> ' + w[::-1] + ' <end>'
    # w = w[::-1]
  return w

In [7]:
ko_sentence = '나는 매일 저녁 배트를 만나러 다락방으로 가요.'
en_sentence = "I go to the attic every evening to meet Bat."
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(ko_sentence))

<start> i go to the attic every evening to meet bat . <end>
<start> . 요가 로으방락다 러나만 를트배 녁저 일매 는나 <end>


In [8]:

def create_dataset(path, num_examples):
  df = pd.read_excel(path,sheet_name=1)
  col = df.columns
  lines = df[col[1]] +'\t'+ df[col[2]]
  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [9]:
en, sp = create_dataset(path_to_file, None)
print(en[-1])
print(sp[-1])

<start> . 죠들힘 이많 엔기기즐 를츠포스 이들이아 린어 <end>
<start> it is difficult for young children to enjoy sports . <end>


In [10]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='',oov_token='<UNK>')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [11]:
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  inp_lang, targ_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

### Limit the size of the dataset to experiment faster (optional)

Training on the complete dataset of >100,000 sentences will take a long time. To train faster, we can limit the size of the dataset to 30,000 sentences (of course, translation quality degrades with less data):

In [12]:
# import pandas as pd
# news_df = pd.read_excel('/content/drive/My Drive/B반/data/kor.xlsx', sheet_name='Sheet1')
# train_df, val_df, test_df = news_df.iloc[:50000, 1:], news_df.iloc[50000:63000, 1:], news_df.iloc[63000:, 1:]

In [13]:
# Try experimenting with the size of that dataset
num_examples = 75000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)
input_tensor, target_tensor = input_tensor[:63000, 1:], target_tensor[:63000, 1:]
xtest_tensor = input_tensor[63000:,1:]
ytest_tensor =target_tensor[63000:,1:]
# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [14]:
input_tensor.shape, target_tensor.shape

((63000, 20), (63000, 20))

In [15]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=13000,random_state=42)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

50000 50000 13000 13000


In [16]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [17]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
4 ----> .
1778 ----> 요나만
8319 ----> 서에꿈
956 ----> 고자
48779 ----> 고덮꼭
24505 ----> 불이
3 ----> <end>

Target Language; index to word mapping
13451 ----> tucked
13 ----> in
87 ----> well
16 ----> and
362 ----> sleep
11 ----> ,
111 ----> see
8 ----> you
13 ----> in
29 ----> your
1336 ----> dreams
4 ----> .
3 ----> <end>


### Create a tf.data dataset

In [18]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [19]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 20]), TensorShape([64, 20]))

In [20]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [21]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 20, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [22]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [23]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 20, 1)


In [24]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [25]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 19232)


## Define the optimizer and the loss function

In [26]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [27]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## Training

1. Pass the *input* through the *encoder* which return *encoder output* and the *encoder hidden state*.
2. The encoder output, encoder hidden state and the decoder input (which is the *start token*) is passed to the decoder.
3. The decoder returns the *predictions* and the *decoder hidden state*.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use *teacher forcing* to decide the next input to the decoder.
6. *Teacher forcing* is the technique where the *target word* is passed as the *next input* to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [28]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [29]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 5.0940
Epoch 1 Batch 100 Loss 2.8624
Epoch 1 Batch 200 Loss 2.9663
Epoch 1 Batch 300 Loss 2.9179
Epoch 1 Batch 400 Loss 2.8892
Epoch 1 Batch 500 Loss 2.4440
Epoch 1 Batch 600 Loss 2.4842
Epoch 1 Batch 700 Loss 2.5433
Epoch 1 Loss 2.7604
Time taken for 1 epoch 305.310898065567 sec

Epoch 2 Batch 0 Loss 2.4916
Epoch 2 Batch 100 Loss 2.4195
Epoch 2 Batch 200 Loss 2.2893
Epoch 2 Batch 300 Loss 2.3182
Epoch 2 Batch 400 Loss 2.3274
Epoch 2 Batch 500 Loss 2.2301
Epoch 2 Batch 600 Loss 2.2703
Epoch 2 Batch 700 Loss 2.2960
Epoch 2 Loss 2.3178
Time taken for 1 epoch 287.53005814552307 sec

Epoch 3 Batch 0 Loss 2.1808
Epoch 3 Batch 100 Loss 2.0597
Epoch 3 Batch 200 Loss 2.1422
Epoch 3 Batch 300 Loss 2.0543
Epoch 3 Batch 400 Loss 2.0468
Epoch 3 Batch 500 Loss 1.8853
Epoch 3 Batch 600 Loss 2.0331
Epoch 3 Batch 700 Loss 2.0593
Epoch 3 Loss 2.0727
Time taken for 1 epoch 285.0877344608307 sec

Epoch 4 Batch 0 Loss 1.8824
Epoch 4 Batch 100 Loss 1.9199
Epoch 4 Batch 200 Loss 1.8455


## Translate

* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token*.
* And store the *attention weights for every time step*.

Note: The encoder output is calculated only once for one input.

In [30]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [31]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [32]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  # print('Input: %s' % (sentence))
  # print('Predicted translation: {}'.format(result))
  return result.replace(' <end>','')

  # attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  # plot_attention(attention_plot, sentence.split(' '), result.split(' '))

## Restore the latest checkpoint and test

In [33]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [34]:
df['en'].iloc[3]

"I'm going back to Korea today at midnight."

In [ ]:
df['en'].iloc[11]

'He leaves for Suwon in a few minutes.'

In [ ]:
df['ko'].iloc[3]

'나는 오늘 자정에 한국으로 돌아 가요.'

In [ ]:
df['ko'].iloc[11]


'그는 조금 있으면 수원으로 가요.'

In [ ]:
translate('어린 아이들이 스포츠를 즐기기엔 많이 힘들죠.')

it is difficult for young children to play sports . <end> 


In [ ]:
translate('나는 오늘 자정에 한국으로 돌아 가요.')

i am going back to korea tonight . <end> 


In [ ]:
translate('그는 조금 있으면 수원으로 가요.')

he leaves for suwon in a little . <end> 


In [ ]:
translate('선생님이 밥을 먹으러 갔다.')


m going to go to my dad and have a younger brother . <end> 


## BLEU SCORE

In [35]:
from tqdm.notebook import tqdm

In [36]:
ref = []
pred = []
for i in tqdm(range(63000,69501)):
  ref.append(df['en'].iloc[i])
  pred.append(translate(df['ko'].iloc[i]))
for i in tqdm(range(69501,74999)):
  ref.append(df['en'].iloc[i])
  pred.append(translate(df['ko'].iloc[i]))

In [39]:
import nltk.translate.bleu_score as bleu
from nltk.translate.bleu_score import SmoothingFunction
def sentences_to_bleu(ref, pred):
  """
  ref : 참고용 타겟 문장(학습용 영어 문장)
  pred : 예측 문장(번역 결과)
  """
  smoothie = SmoothingFunction().method4
  return bleu.sentence_bleu(ref,pred, smoothing_function=smoothie)

In [46]:
pred[0]

'have forgotten that there was there today . '

In [47]:
ref[0]

"And I'm here to explain the reason for this."

In [48]:
score = []
for i in tqdm(range(len(ref))):
  score.append(sentences_to_bleu(ref[i],pred[i]))
  # score.append(sentences_to_bleu(df['en'].iloc[i],translate(df['ko'].iloc[i])))
  # score.append(sentences_to_bleu(ref,pred))
  
# for i in tqdm(range(69001,75000)):
#   score.append(sentences_to_bleu(df['en'].iloc,translate(df['ko'].iloc[i])))
BLEU = np.mean(score)

In [49]:
BLEU

0.31137686954902555